In [1]:
import itertools
from pprint import pformat
from typing import Dict

import pandas as pd
from tqdm.auto import tqdm
from joblib import Parallel, delayed

In [2]:
df = pd.read_feather('../data/master_dataset.ftr')
df

,dhs_id,chr,start,end,DHS_width,summit,numsamples,total_signal,component,proportion,...,fKidney_ENCLB005SRL,fKidney_ENCLB704GMQ,fKidney_ENCLB759USM,fLung_ENCLB594BSZ,fKidney_ENCLB049MNH,fUmbilical_cord_ENCLB771UER,fBone_femur_ENCLB236BWV,fLiver_ENCLB638FEH,fPlacenta_ENCLB423VBC,fPlacenta_ENCLB711ZZZ
0,chr1_16140_16200_16170,chr1,16140,16200,60,16170,1,0.129388,1,0.855153,...,0,0,0,0,0,0,0,0,0,0
1,chr1_51868_52040_51970,chr1,51868,52040,172,51970,1,0.080034,7,0.973545,...,0,0,0,0,0,0,0,0,0,0
2,chr1_57280_57354_57350,chr1,57280,57354,74,57350,4,1.093002,8,1.000000,...,0,0,0,0,0,0,0,0,0,0
3,chr1_66370_66482_66430,chr1,66370,66482,112,66430,8,1.469725,3,0.332213,...,0,0,0,0,0,0,0,0,0,0
4,chr1_79100_79231_79150,chr1,79100,79231,131,79150,2,0.226098,7,0.501840,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591893,chrY_56882540_56882719_56882610,chrY,56882540,56882719,179,56882610,1,0.038079,5,0.803229,...,0,0,0,0,0,0,0,0,0,0
3591894,chrY_56882864_56882980_56882930,chrY,56882864,56882980,116,56882930,1,0.115489,5,0.742349,...,0,0,0,0,0,0,0,0,0,0
3591895,chrY_56883733_56883960_56883830,chrY,56883733,56883960,227,56883830,5,2.456885,7,0.559734,...,0,0,0,0,0,0,0,0,0,0
3591896,chrY_56884440_56884580_56884510,chrY,56884440,56884580,140,56884510,1,0.053759,5,0.803229,...,0,0,0,0,0,0,0,0,0,0


In [3]:
DHS_Index_and_Vocabulary_metadata = pd.read_table('../data/DHS_Index_and_Vocabulary_metadata.tsv').iloc[:-1]
DHS_Index_and_Vocabulary_metadata["Sample ID"] = DHS_Index_and_Vocabulary_metadata[["Biosample name", "DCC Library ID"]].apply(lambda x: "_".join(x), axis=1)
DHS_Index_and_Vocabulary_metadata

,library order,Biosample name,Vocabulary representative,DCC Experiment ID,DCC Library ID,DCC Biosample ID,DCC File ID,Altius Aggregation ID,Altius Library ID,Altius Biosample ID,...,DNaseI units/mL,Amount Nucleic Acid (ng),Nuclei count,Protease inhibitor,Library sequencing date,Reads used,DCC SPOT score,Per-biosample peaks,DHSs in Index,Sample ID
0,1.0,GM06990,NaN,ENCSR000EMQ,ENCLB435ZZZ,ENCBS057ENC,ENCFF983CTQ,AG5636,LN1203,DS7748,...,NaN,50,NaN,NaN,2009-02-23,142681590.0,0.6790,83639.0,82918.0,GM06990_ENCLB435ZZZ
1,2.0,HepG2,NaN,ENCSR000ENP,ENCLB480ZZZ,ENCBS114ENC,ENCFF419JVG,AG5635,LN1207,DS7764,...,NaN,50,NaN,NaN,2009-02-23,138826342.0,0.5858,89748.0,89235.0,HepG2_ENCLB480ZZZ
2,3.0,hTH1,NaN,ENCSR000EQC,ENCLB591ZZZ,ENCBS345AAA,ENCFF575KOF,AG5634,LN1222,DS7840,...,6.0,534.9,NaN,NaN,2007-06-06,149158633.0,0.6470,94360.0,93665.0,hTH1_ENCLB591ZZZ
3,4.0,Hela,NaN,ENCSR000ENO,ENCLB479ZZZ,ENCBS890POO,ENCFF503PAE,AG4219,LN1264,DS8200,...,4.0,50,NaN,NaN,2007-08-24,23372724.0,0.6444,59098.0,59024.0,Hela_ENCLB479ZZZ
4,5.0,CACO2,NaN,ENCSR000EMI,ENCLB422ZZZ,ENCBS391ENC,ENCFF977BRD,AG4218,LN1269,DS8235,...,8.0,1,NaN,NaN,2007-09-05,22760059.0,0.7190,29894.0,29724.0,CACO2_ENCLB422ZZZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,729.0,fUmbilical_cord,NaN,ENCSR512CWR,ENCLB771UER,ENCBS518LEK,ENCFF267RUD,AG7441,LN45036A,DS24820A,...,NaN,0.9,1260000.0,A+Sucrose,2017-02-17,195057523.0,0.5652,113517.0,112469.0,fUmbilical_cord_ENCLB771UER
729,730.0,fBone_femur,Musculoskeletal,ENCSR805XIF,ENCLB236BWV,ENCBS337FPV,ENCFF604WIO,AG7442,LN45038B,DS36206B,...,NaN,8.8,1050000.0,A+Sucrose,2017-02-17,252066174.0,0.5823,146918.0,145356.0,fBone_femur_ENCLB236BWV
730,731.0,fLiver,NaN,ENCSR562FNN,ENCLB638FEH,ENCBS275VNY,ENCFF795ZXN,AG7443,LN45070C,DS37372C,...,NaN,4.48,2140000.0,A+Sucrose,NaN,190541422.0,0.3703,76639.0,75369.0,fLiver_ENCLB638FEH
731,732.0,fPlacenta,NaN,ENCSR552RKI,ENCLB423VBC,ENCBS565KNL,ENCFF084UVH,AG8805,LN45072C,DS37386C,...,NaN,1.325,1050000.0,A+Sucrose,NaN,203699532.0,0.3869,107611.0,106022.0,fPlacenta_ENCLB423VBC


In [4]:
def aggregate_groups_filter_exclusive(groups: Dict[str, str]) -> pd.DataFrame:
    group_ids = list(groups)
    all_ids = list(itertools.chain.from_iterable(groups.values()))
    df_new = df[df[all_ids].any(axis=1)].copy()
    
    agg_dict = {}
    for k, v in (pbar := tqdm(groups.items(), total=len(groups))):
        pbar.set_description(k)
        # Aggregate peaks within a group
        agg_dict[k] = df_new[v].sum(1)
    df_new = pd.concat((df_new, pd.DataFrame(agg_dict)), axis=1)
    
    # Create a new column for each cell type with the exclusive peaks or 'NO_TAG' if not exclusive
    df_new['TAG'] = df_new[group_ids].apply(lambda x: 'NO_TAG' if (x > 0).sum() != 1 else x.idxmax(), axis=1)
    
    # Subset to only exclusive peaks
    n_full_pks = df_new.shape[0]
    df_new = df_new[df_new["TAG"] != "NO_TAG"]
    n_exc_pks = df_new.shape[0]
    print(f"Total number of exclusive peaks: {n_exc_pks:,} ({n_exc_pks / n_full_pks:.2%} of all peaks)")
    
    # Record total peaks and binarize
    df_new["total_peaks"] = df_new[group_ids].sum(1)
    df_new[group_ids] = df_new[group_ids].clip(0, 1)
    
    # Sort by number of exclusive peaks
    df_new = df_new.sort_values("total_peaks", ascending=False, ignore_index=True)

    return df_new

In [5]:
test_group = {"a": ["GM06990_ENCLB435ZZZ", "HepG2_ENCLB480ZZZ"], "b": ["hTH1_ENCLB591ZZZ"]}
aggregate_groups_filter_exclusive(test_group)

  0%|          | 0/2 [00:00<?, ?it/s]

Total number of exclusive peaks: 129,840 (71.13% of all peaks)


,dhs_id,chr,start,end,DHS_width,summit,numsamples,total_signal,component,proportion,...,fKidney_ENCLB049MNH,fUmbilical_cord_ENCLB771UER,fBone_femur_ENCLB236BWV,fLiver_ENCLB638FEH,fPlacenta_ENCLB423VBC,fPlacenta_ENCLB711ZZZ,a,b,TAG,total_peaks
0,chr5_114361011_114361368_114361150,chr5,114361011,114361368,357,114361150,224,316.783950,10,0.207904,...,1,1,1,0,0,0,1,0,a,2
1,chr6_52352440_52352740_52352590,chr6,52352440,52352740,300,52352590,352,1908.978414,2,0.253270,...,0,1,1,0,0,0,1,0,a,2
2,chr16_29645780_29645961_29645870,chr16,29645780,29645961,181,29645870,196,283.645901,4,0.178211,...,0,0,0,0,0,0,1,0,a,2
3,chr4_9153180_9153365_9153250,chr4,9153180,9153365,185,9153250,18,71.078630,1,0.345024,...,0,0,0,0,0,0,1,0,a,2
4,chr15_73053060_73053380_73053210,chr15,73053060,73053380,320,73053210,425,517.172261,1,0.164268,...,1,1,1,0,0,0,1,0,a,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129835,chr16_30188100_30188341_30188250,chr16,30188100,30188341,241,30188250,17,5.821924,5,0.797315,...,0,0,0,0,0,0,0,1,b,1
129836,chr16_30187880_30188160_30188020,chr16,30187880,30188160,280,30188020,10,5.084031,5,0.506208,...,0,0,0,0,0,0,0,1,b,1
129837,chr16_30184500_30184760_30184630,chr16,30184500,30184760,260,30184630,101,135.293503,5,0.315205,...,1,0,1,0,0,0,0,1,b,1
129838,chr16_30182796_30183061_30182930,chr16,30182796,30183061,265,30182930,200,137.957977,3,0.199071,...,1,1,0,0,1,1,0,1,b,1


### Group by cell types

In [6]:
cell_type_groups = {
    i: j["Sample ID"].tolist() for i, j in DHS_Index_and_Vocabulary_metadata[
        (DHS_Index_and_Vocabulary_metadata["Biological state"] == "Primary")
        & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
    ][["Sample ID", "Biosample name"]].groupby("Biosample name")
}
cell_type_groups

{'AG09309': ['AG09309_ENCLB406ZZZ', 'AG09309_ENCLB407ZZZ'],
 'AG09319': ['AG09319_ENCLB408ZZZ', 'AG09319_ENCLB409ZZZ'],
 'AG10803': ['AG10803_ENCLB410ZZZ', 'AG10803_ENCLB411ZZZ'],
 'AoAF': ['AoAF_ENCLB412ZZZ', 'AoAF_ENCLB413ZZZ'],
 'Brain_Cerebellar_Cortex': ['Brain_Cerebellar_Cortex_ENCLB199OZA'],
 'Brain_Globus_Pallidus': ['Brain_Globus_Pallidus_ENCLB853MWO'],
 'Brain_Mid_Frontal_Gyrus': ['Brain_Mid_Frontal_Gyrus_ENCLB524IQB'],
 'Brain_Putamen': ['Brain_Putamen_ENCLB558ETM'],
 'Brain_hippocampus': ['Brain_hippocampus_ENCLB763IVQ'],
 'CD14': ['CD14_ENCLB665NTX',
  'CD14_ENCLB238BXL',
  'CD14_ENCLB885QAS',
  'CD14_ENCLB556ZZZ'],
 'CD19': ['CD19_ENCLB800FZO', 'CD19_ENCLB894REI', 'CD19_ENCLB534RFX'],
 'CD20': ['CD20_ENCLB424ZZZ', 'CD20_ENCLB425ZZZ'],
 'CD3': ['CD3_ENCLB092RHM', 'CD3_ENCLB855KQP', 'CD3_ENCLB597AVO'],
 'CD34': ['CD34_ENCLB262ADG',
  'CD34_ENCLB623PEA',
  'CD34_ENCLB786RJZ',
  'CD34_ENCLB794TRG',
  'CD34_ENCLB323HYV',
  'CD34_ENCLB336NTC',
  'CD34_ENCLB860SDJ'],
 'CD4': ['C

In [7]:
df_cell_type = aggregate_groups_filter_exclusive(cell_type_groups)
# df_cell_type.to_feather("../data/processed/cell_type_cls/filtered_exclusive.ftr")  # uncomment to save
df_cell_type

  0%|          | 0/56 [00:00<?, ?it/s]

Total number of exclusive peaks: 519,095 (38.99% of all peaks)


,dhs_id,chr,start,end,DHS_width,summit,numsamples,total_signal,component,proportion,...,hTH17,hTH2,hTR,iTH1,iTH2,kidney_tubular,renal_cell_carcinoma,vHMEC,TAG,total_peaks
0,chr1_170041900_170042138_170042010,chr1,170041900,170042138,238,170042010,29,75.292284,15,0.933175,...,0,0,0,0,0,0,0,0,CD34,7
1,chr10_47306420_47306660_47306550,chr10,47306420,47306660,240,47306550,30,69.559214,15,0.915566,...,0,0,0,0,0,0,0,0,CD34,7
2,chr6_164042320_164042580_164042450,chr6,164042320,164042580,260,164042450,34,225.387994,15,0.879957,...,0,0,0,0,0,0,0,0,CD34,7
3,chr22_27612165_27612580_27612360,chr22,27612165,27612580,415,27612360,33,244.434882,15,0.808165,...,0,0,0,0,0,0,0,0,CD34,7
4,chr4_59560220_59560480_59560350,chr4,59560220,59560480,260,59560350,51,175.954764,3,0.513419,...,0,0,0,0,0,0,0,0,CD34,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519090,chr16_52311941_52312020_52311990,chr16,52311941,52312020,79,52311990,1,0.245914,13,0.869445,...,0,0,0,0,0,0,0,0,Small_Intestine_Mucosa,1
519091,chr16_52311546_52311820_52311690,chr16,52311546,52311820,274,52311690,3,1.033634,13,0.964817,...,0,0,0,0,0,0,0,0,Small_Intestine_Mucosa,1
519092,chr16_52310997_52311193_52311090,chr16,52310997,52311193,196,52311090,2,0.560019,13,0.947795,...,0,0,0,0,0,0,0,0,Small_Intestine_Mucosa,1
519093,chr16_52310140_52310340_52310230,chr16,52310140,52310340,200,52310230,11,6.212531,3,0.902571,...,0,0,0,0,0,1,0,0,kidney_tubular,1


### Group by tissues

In [8]:
tissue_groups = {
    i: j["Sample ID"].tolist() for i, j in DHS_Index_and_Vocabulary_metadata[
        (DHS_Index_and_Vocabulary_metadata["Biological state"] == "Primary")
        & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
    ][["Sample ID", "Organ"]].groupby("Organ")
}
tissue_groups

{'Blood': ['hTH1_ENCLB591ZZZ',
  'hTH17_ENCLB597ZZZ',
  'CD34_ENCLB262ADG',
  'CD34_ENCLB623PEA',
  'CD34_ENCLB786RJZ',
  'CD4pos_N_ENCLB427ZZZ',
  'CD34_ENCLB794TRG',
  'CD34_ENCLB323HYV',
  'CD34_ENCLB336NTC',
  'hTR_ENCLB602ZZZ',
  'CD4_ENCLB354KDT',
  'CD56_ENCLB529QSB',
  'CD3_ENCLB092RHM',
  'CD4_ENCLB056XEC',
  'CD8_ENCLB979DCJ',
  'CD4_ENCLB135TML',
  'CD19_ENCLB800FZO',
  'CD56_ENCLB968GLJ',
  'CD3_ENCLB855KQP',
  'CD8_ENCLB798GQN',
  'CD14_ENCLB665NTX',
  'CD19_ENCLB894REI',
  'CD4_ENCLB882RDH',
  'CD8_ENCLB060BVC',
  'CD14_ENCLB238BXL',
  'CD19_ENCLB534RFX',
  'CD20_ENCLB424ZZZ',
  'hTH1_ENCLB595ZZZ',
  'hTH2_ENCLB601ZZZ',
  'iTH2_ENCLB598ZZZ',
  'CD4_ENCLB175IVE',
  'CD14_ENCLB885QAS',
  'hTH1_ENCLB594ZZZ',
  'iTH1_ENCLB592ZZZ',
  'CD14_ENCLB556ZZZ',
  'CD20_ENCLB425ZZZ',
  'CD34_ENCLB860SDJ',
  'CD8_ENCLB305XZE',
  'CD4_ENCLB270QXD',
  'Dendritic_cells_C51_ENCLB334ICF',
  'Dendritic_cells_C190_ENCLB958JIU',
  'hTH2_ENCLB599ZZZ'],
 'Brain': ['Brain_hippocampus_ENCLB763IVQ',

In [9]:
df_tissue = aggregate_groups_filter_exclusive(tissue_groups)
# df_tissue.to_feather("../data/processed/tissue_cls/filtered_exclusive.ftr")  # uncomment to save
df_tissue

  0%|          | 0/14 [00:00<?, ?it/s]

Total number of exclusive peaks: 683,222 (51.32% of all peaks)


,dhs_id,chr,start,end,DHS_width,summit,numsamples,total_signal,component,proportion,...,Mammary,Mesoderm,Ovary,Pancreas,Prostate,Skin,Small Intestine,Stroma,TAG,total_peaks
0,chr18_62869720_62869920_62869830,chr18,62869720,62869920,200,62869830,67,117.306242,5,0.575278,...,0,0,0,0,0,0,0,0,Blood,35
1,chr15_94287871_94288080_94287970,chr15,94287871,94288080,209,94287970,91,406.019704,5,0.495548,...,0,0,0,0,0,0,0,0,Blood,35
2,chr22_40400240_40400410_40400330,chr22,40400240,40400410,170,40400330,94,286.775880,5,0.543945,...,0,0,0,0,0,0,0,0,Blood,34
3,chr5_16629463_16629700_16629590,chr5,16629463,16629700,237,16629590,99,287.478511,5,0.304142,...,0,0,0,0,0,0,0,0,Blood,34
4,chr2_145411591_145411820_145411690,chr2,145411591,145411820,229,145411690,93,193.505799,5,0.530140,...,0,0,0,0,0,0,0,0,Blood,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683217,chr17_72423802_72424020_72423910,chr17,72423802,72424020,218,72423910,6,10.067674,5,1.000000,...,0,0,0,0,0,0,0,0,Blood,1
683218,chr17_72422880_72423360_72423230,chr17,72422880,72423360,480,72423230,1,0.158902,5,1.000000,...,0,0,0,0,0,0,0,0,Blood,1
683219,chr17_72413260_72413620_72413390,chr17,72413260,72413620,360,72413390,30,144.674764,13,0.912903,...,0,0,0,0,0,0,1,0,Small Intestine,1
683220,chr17_72409760_72409928_72409850,chr17,72409760,72409928,168,72409850,20,9.177497,15,0.568687,...,0,0,0,0,0,0,0,0,Blood,1


### Group by cancer (kidney tubular)

Only 'kidney tubular' and 'renal cell carcinoma' cell type are shared between 'Cancer' and 'Primary' groups in adult samples. We will focus on the 'kidney tubular' cells.

In [10]:
cancer_celltype_set = set(
    DHS_Index_and_Vocabulary_metadata[
        (DHS_Index_and_Vocabulary_metadata["Biological state"] == "Cancer")
        & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
    ]["Biosample name"].value_counts().index.tolist()
) & set(
    DHS_Index_and_Vocabulary_metadata[
        (DHS_Index_and_Vocabulary_metadata["Biological state"] == "Primary")
        & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
    ]["Biosample name"].value_counts().index.tolist()
)
cancer_celltype_set

{'kidney_tubular', 'renal_cell_carcinoma'}

In [11]:
DHS_Index_and_Vocabulary_metadata[
    DHS_Index_and_Vocabulary_metadata["Biosample name"].isin(cancer_celltype_set)
    & (DHS_Index_and_Vocabulary_metadata["Biological state"].isin(["Cancer", "Primary"]))
    & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
][["Biosample name", "Biological state"]].value_counts()

Biosample name        Biological state
kidney_tubular        Primary             3
renal_cell_carcinoma  Cancer              3
kidney_tubular        Cancer              1
renal_cell_carcinoma  Primary             1
Name: count, dtype: int64

In [12]:
cancer_kidney_tubular_groups = {
    i: j["Sample ID"].tolist() for i, j in DHS_Index_and_Vocabulary_metadata[
        (DHS_Index_and_Vocabulary_metadata["Biological state"].isin(["Cancer", "Primary"]))
        & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
        & (DHS_Index_and_Vocabulary_metadata["Biosample name"] == "kidney_tubular")
    ][["Sample ID", "Biological state"]].groupby("Biological state")
}
cancer_kidney_tubular_groups

{'Cancer': ['kidney_tubular_ENCLB108PBP'],
 'Primary': ['kidney_tubular_ENCLB997AWQ',
  'kidney_tubular_ENCLB626YLU',
  'kidney_tubular_ENCLB888MCL']}

In [13]:
df_cancer_kidney_tubular = aggregate_groups_filter_exclusive(cancer_kidney_tubular_groups)
# df_cancer_kidney_tubular.to_feather("../data/processed/cancer_kidney_tubular_cls/filtered_exclusive.ftr")  # uncomment to save
df_cancer_kidney_tubular

  0%|          | 0/2 [00:00<?, ?it/s]

Total number of exclusive peaks: 116,381 (39.81% of all peaks)


,dhs_id,chr,start,end,DHS_width,summit,numsamples,total_signal,component,proportion,...,fKidney_ENCLB049MNH,fUmbilical_cord_ENCLB771UER,fBone_femur_ENCLB236BWV,fLiver_ENCLB638FEH,fPlacenta_ENCLB423VBC,fPlacenta_ENCLB711ZZZ,Cancer,Primary,TAG,total_peaks
0,chr1_629160_629310_629230,chr1,629160,629310,150,629230,281,11367.849053,2,0.140686,...,0,0,0,0,0,0,0,1,Primary,3
1,chr6_146357834_146358080_146357950,chr6,146357834,146358080,246,146357950,222,616.087961,2,0.281134,...,0,0,0,0,0,0,0,1,Primary,3
2,chr2_215454559_215454740_215454630,chr2,215454559,215454740,181,215454630,218,156.182922,4,0.165287,...,0,0,0,0,0,0,0,1,Primary,3
3,chr2_215799300_215799478_215799390,chr2,215799300,215799478,178,215799390,53,54.104149,4,0.382397,...,0,0,0,0,0,0,0,1,Primary,3
4,chr14_25537940_25538120_25538050,chr14,25537940,25538120,180,25538050,245,178.801127,3,0.236966,...,1,0,0,0,0,0,0,1,Primary,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116376,chr18_2446660_2446840_2446750,chr18,2446660,2446840,180,2446750,4,2.166291,6,0.674747,...,0,0,0,0,0,0,0,1,Primary,1
116377,chr18_2430536_2430654_2430590,chr18,2430536,2430654,118,2430590,7,2.026900,16,0.803681,...,0,0,0,0,0,0,0,1,Primary,1
116378,chr18_2410520_2410645_2410620,chr18,2410520,2410645,125,2410620,4,1.865601,6,0.396922,...,0,0,0,0,0,0,0,1,Primary,1
116379,chr18_2396306_2396420_2396350,chr18,2396306,2396420,114,2396350,2,0.352040,6,0.980812,...,0,0,0,0,0,0,0,1,Primary,1


### Exploring cancer grouping by tissue

Blood tissue contains many samples for both the 'Cancer' and 'Primary' groups. But the cell types are too heterogeneous and might be mistleading to evaluate against, as the distinctive features could come from  cell type differences.

In [14]:
cancer_tissue_set = set(
    DHS_Index_and_Vocabulary_metadata[
        (DHS_Index_and_Vocabulary_metadata["Biological state"] == "Cancer")
        & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
    ]["Organ"].value_counts().index.tolist()
) & set(
    DHS_Index_and_Vocabulary_metadata[
        (DHS_Index_and_Vocabulary_metadata["Biological state"] == "Primary")
        & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
    ]["Organ"].value_counts().index.tolist()
)
cancer_tissue_set

{'Blood', 'Brain', 'Kidney', 'Lung', 'Mammary', 'Mesoderm', 'Prostate', 'Skin'}

In [15]:
DHS_Index_and_Vocabulary_metadata[
    DHS_Index_and_Vocabulary_metadata["Organ"].isin(cancer_tissue_set)
    & (DHS_Index_and_Vocabulary_metadata["Biological state"].isin(["Cancer", "Primary"]))
    & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
][["Organ", "Biological state"]].value_counts()

Organ     Biological state
Blood     Primary             42
          Cancer              18
Skin      Primary             17
Mammary   Cancer              11
Kidney    Cancer              10
Lung      Primary             10
Kidney    Primary              9
Brain     Primary              7
Lung      Cancer               6
Mammary   Primary              6
Brain     Cancer               5
Prostate  Cancer               4
Skin      Cancer               4
Mesoderm  Cancer               2
Prostate  Primary              2
Mesoderm  Primary              1
Name: count, dtype: int64

In [16]:
cancer_blood_groups = {
    i: j["Sample ID"].tolist() for i, j in DHS_Index_and_Vocabulary_metadata[
        (DHS_Index_and_Vocabulary_metadata["Biological state"].isin(["Cancer", "Primary"]))
        & (DHS_Index_and_Vocabulary_metadata["Growth stage"] == "Adult")
        & (DHS_Index_and_Vocabulary_metadata["Organ"] == "Blood")
    ][["Sample ID", "Biological state"]].groupby("Biological state")
}
cancer_blood_groups

{'Cancer': ['K562_ENCLB539ZZZ',
  'K562_ENCLB540ZZZ',
  'NB4_ENCLB557ZZZ',
  'NB4_ENCLB558ZZZ',
  'K562_ENCLB253REF',
  'K562_ENCLB843GMH',
  'RPMI_8226_ENCLB167OQH',
  'KBM7_ENCLB182IDD',
  'KBM7_ENCLB484IPH',
  'MM_1S_ENCLB793GPI',
  'Karpas_422_ENCLB246IPC',
  'Karpas_422_ENCLB613MEH',
  'Oci_Ly_7_ENCLB763ZVD',
  'Oci_Ly_7_ENCLB753TXB',
  'RPMI_8226_ENCLB123CIH',
  'HAP1_ENCLB760TOD',
  'HAP1_ENCLB949CJA',
  'K562_ENCLB096YUZ'],
 'Primary': ['hTH1_ENCLB591ZZZ',
  'hTH17_ENCLB597ZZZ',
  'CD34_ENCLB262ADG',
  'CD34_ENCLB623PEA',
  'CD34_ENCLB786RJZ',
  'CD4pos_N_ENCLB427ZZZ',
  'CD34_ENCLB794TRG',
  'CD34_ENCLB323HYV',
  'CD34_ENCLB336NTC',
  'hTR_ENCLB602ZZZ',
  'CD4_ENCLB354KDT',
  'CD56_ENCLB529QSB',
  'CD3_ENCLB092RHM',
  'CD4_ENCLB056XEC',
  'CD8_ENCLB979DCJ',
  'CD4_ENCLB135TML',
  'CD19_ENCLB800FZO',
  'CD56_ENCLB968GLJ',
  'CD3_ENCLB855KQP',
  'CD8_ENCLB798GQN',
  'CD14_ENCLB665NTX',
  'CD19_ENCLB894REI',
  'CD4_ENCLB882RDH',
  'CD8_ENCLB060BVC',
  'CD14_ENCLB238BXL',
  'CD19_